In [1]:
path = './zomato.csv'

In [2]:
import csv
import os
import glob
import numpy as np
import math
#scikit learn
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def cleaning(sentence):
    sentence = sentence.replace('\ufeff','')
    sentence = sentence.replace('\n',' ')
    sentence = sentence.replace('\r',' ')
    sentence = sentence.replace('\n\n',' ')
    sentence = sentence.replace('\r\r',' ')
    sentence = sentence.replace('.',' ')
    sentence = sentence.replace(',',' ')
    sentence = sentence.replace('"',' ')
    sentence = sentence.replace('!',' ')
    sentence = sentence.replace('?',' ')
    sentence = sentence.replace('/',' ')
    sentence = sentence.replace('-',' ')
    sentence = sentence.lstrip()
    return sentence.lower()

In [4]:
def convert(sentences):
    import re
    cleanEmoticon = []
    for sentence in sentences:
        try:
            highpoints = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
        except re.error:
            highpoints = re.compile(u'([\u2600-\u27BF])|([\uD83C][\uDF00-\uDFFF])|([\uD83D][\uDC00-\uDE4F])|([\uD83D][\uDE80-\uDEFF])')
    
        cleanEmoticon.append(highpoints.sub(u'\u25FD', sentence))
    return cleanEmoticon

In [5]:
x = []
y = []
with open(path, encoding="utf8") as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])
x = np.array(x)
y = np.array(y)

In [6]:
randomize = np.arange(len(x))
np.random.shuffle(randomize)
x = x[randomize]
y = y[randomize]

In [7]:
x_train = x[:150]
y_train = y[:150]

x_test = x[150:]
y_test = y[150:]

In [8]:
def preprocess(sentences): 
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    result =[]
    for sentence in sentences:
        result.append(stemmer.stem(stopword.remove(sentence)))
    return result

In [9]:
# stem_test = ['dia biasanya makan nasi padang', 
#              'aku menyukainya dengan tulus dan raisa', 
#              'ku tak kan pernah memilikinya sampai kapanpun', 
#              'karena dia sudah memilikinya & dimilikinya']

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [10]:
x_train = convert(x_train)
x_test = convert(x_test)

### Scikit-Learn Tweaks
Reference:
https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [11]:
from sklearn import svm

text_clf = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', svm.SVC(kernel='linear')),
])
text_clf = text_clf.fit(x_train, y_train)

In [12]:
y_pred = text_clf.predict(x_test)

In [13]:
accuracy_score(y_test, y_pred)

0.8

### Gensim
http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

https://github.com/RaRe-Technologies/gensim/issues/1095

https://markhneedham.com/blog/2018/05/19/interpreting-word2vec-glove-embeddings-sklearn-neo4j-graph-algorithms/

In [14]:
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_20newsgroups
import re
import matplotlib.pyplot as plt

# download example data ( may take a while)
train = fetch_20newsgroups()

def clean(text):
    """Remove posting header, split by sentences and words, keep only letters"""
    lines = re.split('[?!.:]\s', re.sub('^.*Lines: \d+', '', re.sub('\n', ' ', text)))
    return [re.sub('[^a-zA-Z]', ' ', line).lower().split() for line in lines]

sentences = [line for text in train.data for line in clean(text)]

model = Word2Vec(sentences, workers=4, size=100, min_count=50, window=10, sample=1e-3)

print (model.most_similar('memory'))

X = model[model.wv.vocab]

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)



/Users/indraaristya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/indraaristya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/Users/indraaristya/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/Users/indraaristya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[('cpu', 0.8340690732002258), ('cache', 0.7723075151443481), ('monitor', 0.7459338903427124), ('ram', 0.743175208568573), ('configuration', 0.7363064289093018), ('processor', 0.7340117692947388), ('timing', 0.7270035743713379), ('video', 0.726527214050293), ('motherboard', 0.7247623205184937), ('monitors', 0.7234789133071899)]


In [15]:
# download example data ( may take a while)
train = fetch_20newsgroups()

def clean(text):
    """Remove posting header, split by sentences and words, keep only letters"""
    lines = re.split('[?!.:]\s', re.sub('^.*Lines: \d+', '', re.sub('\n', ' ', text)))
    return [re.sub('[^a-zA-Z]', ' ', line).lower().split() for line in lines]

sentences = [line for text in train.data for line in clean(text)]

In [16]:
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(x_train, y_train)

In [17]:
y_pred = text_clf.predict(x_test)

In [18]:
accuracy_score(y_test, y_pred) 

0.84

In [19]:
s_l = classification_report(y_test, y_pred,target_names=['0','1'])

In [20]:
print(s_l)

              precision    recall  f1-score   support

           0       0.78      0.91      0.84        23
           1       0.91      0.78      0.84        27

   micro avg       0.84      0.84      0.84        50
   macro avg       0.85      0.85      0.84        50
weighted avg       0.85      0.84      0.84        50



In [21]:
confusion_matrix(y_test, y_pred)

array([[21,  2],
       [ 6, 21]])

In [22]:
from sklearn.model_selection import KFold

x = np.concatenate((x_train,x_test))
y = np.concatenate((y_train,y_test))
        
randomize = np.arange(len(x))
np.random.shuffle(randomize)

acc = []
folds = 5
kf = KFold(n_splits=folds)
for train, test in kf.split(x):
    x_train, x_test, y_train, y_test = x[train],x[test],y[train],y[test]
    
    text_clf = Pipeline([('vect', CountVectorizer()),
                     ('clf', MultinomialNB()),
    ])
    text_clf = text_clf.fit(x_train, y_train)
    y_pred = text_clf.predict(x_test)
    acc.append(accuracy_score(y_test, y_pred))
print("Accuracy:",acc)
print("Mean Acc of",folds,"Fold:",np.mean(acc))

Accuracy: [0.725, 0.825, 0.825, 0.725, 0.8]
Mean Acc of 5 Fold: 0.78
